In [53]:
# ! pip install tensorflow

In [54]:
# Importação de bibliotecas
import numpy as np
import pandas as pd
import tensorflow as tf

In [55]:
# Versão do códio
tf.__version__

'2.5.0'

In [56]:
# Importando base de dados
dataset = pd.read_csv('Churn_Modelling.csv')

In [57]:
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [58]:
# Lendo 
#dataset.Geography.value_counts()

In [59]:
# Exemplo
# x = dataset.iloc[:,1:2].values # Essa função pega todos os valores da coluna 1 até a coluna 2
# y = dataset.iloc[:,2].values # Essa função pega todos os valores da coluna 2


In [60]:
X  = dataset.iloc[:, 3:13].values # Essa função pega todos valores de todas as colunas de 2 até 12
y = dataset.iloc[:, 13].values # Essa função pega todos valores da coluna 13
print("X ..\n",X)
print("y ..\n",y)

X ..
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y ..
 [1 0 1 ... 1 1 0]


In [61]:
# Transformando dados categóricos em binários
# coluna de gênenro ( 2 + 3 )
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# coluna Geography ( 2 + 3 )
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [62]:
# Dividindo o dataser em conjunto de treinamento e conjunto de teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [63]:
# Feature Scaling  ajustando escalas
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [64]:
np.shape(X_train)

(8000, 12)

In [65]:
#X_train

In [66]:
# Construção ANN!

In [67]:
ann = tf.keras.models.Sequential()

In [68]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))

In [69]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [70]:
ann.add(tf.keras.layers.Dense(units=9,activation='relu'))

In [71]:
ann.add(tf.keras.layers.Dense(units=12,activation='relu'))

In [72]:
ann.add(tf.keras.layers.Dense(units=9,activation='relu'))

In [73]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [74]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))

In [75]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [76]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [77]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6179 - accuracy: 0.7805
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4765 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4578 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4463 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4384 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4321 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4257 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4167 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4141 - accura

# Predições e avalliação do modelo

In [78]:
y_pred = ann.predict(X_test)
y_pred

array([[0.2308903 ],
       [0.39075646],
       [0.14034072],
       ...,
       [0.14879352],
       [0.12613696],
       [0.24467185]], dtype=float32)

In [79]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [80]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1541   54]
 [ 206  199]]


In [81]:
# Acurácia foi de 86.2% para 87%